In [285]:
import requests
import contextlib
import bs4
import re
import collections

In [22]:
URL = r"http://prac.im.pwr.wroc.pl/~hugo/HSC/Publications.html"

In [24]:
with requests.get(url=URL) as response:
    print(response.status_code)
    html = response.content

200


In [26]:
html = bs4.BeautifulSoup(html)

In [73]:
list(html.b.children)

[<a href="#Books"><font color="black" face="Verdana" size="2">Books</font></a>]

In [101]:
for h3 in html.find_all("h3"):
    print(h3.contents)
    print()

['\n', <font color="black" face="Verdana" size="3">2020</font>, '\n']

['\n', <font color="black" face="Verdana" size="3">2019</font>, '\n']

['\n', <font color="black" face="Verdana" size="3">2018</font>, '\n']

['\n', <font color="black" face="Verdana" size="3">2017</font>, '\n']

['\n', <font color="black" face="Verdana" size="3">2016</font>, '\n']

['\n', <font color="black" face="Verdana" size="3">2015</font>, '\n']

['\n', <font color="black" face="Verdana" size="3">2014</font>, '\n']

['\n', <font color="black" face="Verdana" size="3">2013</font>, '\n']

['\n', <font color="black" face="Verdana" size="3">2012</font>, '\n']

['\n', <font color="black" face="Verdana" size="3">2011</font>, '\n']

['\n', <font color="black" face="Verdana" size="3">2010</font>, '\n']

['\n', <font color="black" face="Verdana" size="3">2009</font>, '\n']

['\n', <font color="black" face="Verdana" size="3">2008</font>, '\n']

['\n', <font color="black" face="Verdana" size="3">2007</font>, '\n']

['\n',

In [171]:
import regex

In [307]:
re.match(r'^".+"$', '"a"')

<re.Match object; span=(0, 3), match='"a"'>

In [306]:
def content_type(string: str) -> str:
    """bla bla bla"""
    string = string.strip()
#     if regex.match(r'^[A-Z]', string):
#         return 'test'
    if regex.match(r'^(\pL[.][ ]?)+\pL[a-z]+', string):
        return 'name'
    elif regex.match(r'^[\d]{4}$', string):
        return 'year'
    elif regex.match(r'^".+"$', string):
        return 'title'
    else:
        return 'other'

In [280]:
content_type('A. B. Aak')

'name'

In [281]:
content_type('1234')

'year'

In [302]:
content_type('"aloha"')

'title'

In [301]:
content_type('Alice has a cat.')

'other'

In [313]:
year_articles = collections.defaultdict()

In [316]:
for ol in html.find_all('ol', {'type': '1', 'start': '1'}):
    for li in ol.find_all('li'):
        article_data = collections.defaultdict(list)
        for font in li.find_all('font'):
            content = font.string
            type_ = content_type(content)
            if type_ == 'name':
                article_data['authors'].append(content)
            elif type_ == 'title':
                article_data['title'] = content
#             print(font.string, type_)
        print(article_data['authors'])
        print(article_data['title'])
        print()

['F.Chaari', 'J. Leskow', 'R.Zimroz', 'A.Wylomanska', 'A.Dudek']
"Applied Condition Monitoring. Cyclostationarity: Theory and Methods - IV"

['F.Chaari', 'J. Leskow', 'A. Napolitano', 'R.Zimroz', 'A.Wylomanska']
"Applied Condition Monitoring. Cyclostationarity: Theory and Methods - III"

['R.G.Burns', 'I.Szymaniec', 'A.Weron']
" Hugo Steinhaus Mathematician for All Seasons. Recollections and Notes  Vol.2 (1945-1968)"

['R.G.Burns', 'I.Szymaniec', 'A.Weron']
" Hugo Steinhaus Mathematician for All Seasons. Recollections and Notes  Vol.1 (1887-1945)"

['F.Chaari', 'J. Leskow', 'A. Napolitano', 'R.Zimroz', 'A.Wylomanska', 'A.Dudek']
"Applied Condition Monitoring. Cyclostationarity: Theory and Methods - II"

['K.Burnecki']
[]

['P.Cizek', 'R.Weron']
"Statistical Tools for Finance and Insurance (2 Edition)"

['R.Weron']
"Modeling and Forecasting Electricity Loads and Prices: A Statistical Approach"

['P.Cizek', 'R.Weron']
"Statistical Tools for Finance and Insurance"

['A. Izydorczyk', 'A.Ja

['K.Burnecki', 'A.Misiorek', 'R.Weron']
"Loss distributions"

['K.Burnecki', 'R.Weron']
"Modeling of the risk process"

['K.Burnecki', 'J.Nowicka-Zagrajek ', ' A. Wyłomańska']
"Pure risk premiums under deductibles"

['K.Burnecki', 'G. Kukla', 'D.Taylor']
"Pricing of catastrophe bonds"

['K.Burnecki', 'P. Miśta', 'A.Weron']
"Ruin probabilities in finite and infinite time"

['K.Burnecki', 'P.Miśta', 'A.Weron']
"What is the best approximation of the ruin probability in infinite time?"

['K.Burnecki', 'J.Nowicka-Zagrajek ']
[]

['H.Furrer', 'Z.Michna', 'A.Weron']
"Stable diffusion approximation of the risk process"

['J.Iwanik', 'J.Nowicka-Zagrajek ']
"Premiums in the individual and collective risk models"

['A. Jurlewicz']
"Limit theorems for randomly coarse grained continous-time random walks"

['A.Jurlewicz', 'K. Weron']
"Scaling properties of the diffusion process underlying the Havrilak-Negami relaxation function"

['A.Jurlewicz', 'B. Szabat']
"Temperature dependent cluster model of t

AttributeError: 'NoneType' object has no attribute 'strip'

In [75]:
for a in html.find_all('a'):
    print(list(a.children))

[<font color="black" face="Verdana" size="2">Books</font>]
[<font color="black" face="Verdana" size="2">Theses</font>]
[<font color="black" face="Verdana" size="2">Research papers</font>]
[<font color="black" face="Verdana" size="2">Research reports</font>]
['Books:\r\n     ']
[<font color="black" face="Verdana" size="2"><i>"Applied Condition Monitoring. Cyclostationarity: Theory and Methods - IV"</i></font>]
[<font color="black" face="Verdana" size="2"><i>"Applied Condition Monitoring. Cyclostationarity: Theory and Methods - III"</i></font>]
[<font color="black" face="Verdana" size="2"><i>" Hugo Steinhaus Mathematician for All Seasons. Recollections and Notes  Vol.2 (1945-1968)"</i></font>]
[<font color="black" face="Verdana" size="2"><i>" Hugo Steinhaus Mathematician for All Seasons. Recollections and Notes  Vol.1 (1887-1945)"</i></font>]
[<font color="black" face="Verdana" size="2"><i>"Applied Condition Monitoring. Cyclostationarity: Theory and Methods - II"</i></font>]
[<font color

In [47]:
html.find("a", attrs={'name': '2020'})

<a name="2020"><h3 align="center">
<font color="black" face="Verdana" size="3">2020</font>
</h3></a>

In [33]:
html.select("h3")

[<h3 align="center">
 <font color="black" face="Verdana" size="3">2020</font>
 </h3>, <h3 align="center">
 <font color="black" face="Verdana" size="3">2019</font>
 </h3>, <h3 align="center">
 <font color="black" face="Verdana" size="3">2018</font>
 </h3>, <h3 align="center">
 <font color="black" face="Verdana" size="3">2017</font>
 </h3>, <h3 align="center">
 <font color="black" face="Verdana" size="3">2016</font>
 </h3>, <h3 align="center">
 <font color="black" face="Verdana" size="3">2015</font>
 </h3>, <h3 align="center">
 <font color="black" face="Verdana" size="3">2014</font>
 </h3>, <h3 align="center">
 <font color="black" face="Verdana" size="3">2013</font>
 </h3>, <h3 align="center">
 <font color="black" face="Verdana" size="3">2012</font>
 </h3>, <h3 align="center">
 <font color="black" face="Verdana" size="3">2011</font>
 </h3>, <h3 align="center">
 <font color="black" face="Verdana" size="3">2010</font>
 </h3>, <h3 align="center">
 <font color="black" face="Verdana" size="3"

In [32]:
html.find_all(string=re.compile(r"^[0-9]{4}$"))

['2020',
 '2017',
 '2016',
 '2015',
 '2015',
 '2012',
 '2011',
 '2006',
 '2005',
 '2001',
 '2000',
 '1996',
 '1994',
 '2018',
 '2018',
 '2018',
 '2018',
 '2018',
 '2018',
 '2018',
 '2014',
 '2013',
 '2013',
 '2012',
 '2012',
 '2011',
 '2007',
 '2006',
 '2006',
 '2006',
 '2006',
 '2006',
 '2002',
 '1998',
 '1998',
 '1998',
 '1997',
 '1996',
 '1991',
 '1990',
 '2020',
 '2019',
 '2018',
 '2017',
 '2016',
 '2015',
 '2014',
 '2013',
 '2012',
 '2011',
 '2010',
 '2009',
 '2008',
 '2007',
 '2006',
 '2005',
 '2004',
 '2003',
 '2002',
 '2001',
 '2000',
 '1999',
 '1998',
 '1997',
 '1996',
 '1995',
 '2020',
 '2020',
 '2020',
 '2020',
 '2020',
 '2019',
 '2019',
 '2019',
 '2019',
 '2019',
 '2019',
 '2019',
 '2019',
 '2019',
 '2019',
 '2019',
 '2019',
 '2019',
 '2019',
 '2019',
 '2019',
 '2019',
 '2019',
 '2019',
 '2019',
 '2019',
 '2019',
 '2019',
 '2019',
 '2019',
 '2019',
 '2019',
 '2019',
 '2019',
 '2018',
 '2018',
 '2018',
 '2018',
 '2018',
 '2018',
 '2018',
 '2018',
 '2018',
 '2018',
 '2018',
 

In [2]:
def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of resposnse is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        with contextlib.closing(requests.get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None
    except requests.exceptions.RequestException as e:
        log_error(f"Error during requests to {url}: {str(e)}")
        return None

In [8]:
def is_good_response(resp):
    """
    Returns True if the resposne seems to be HTML, False otherwise.
    """
    content_type = resp.headers["Content-Type"].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find("html") > -1)

In [6]:
def log_error(e):
    """
    It is always a good idea to log errors.
    This function just prints them, but you can
    make it do anything.
    """
    print(e)

In [18]:
simple_get("http://bajogrodek.pl")

b'\xef\xbb\xbf<html>\n<head>\n    <meta http-equiv="content-type" content="text/html" charset="UTF-8">\n    <meta name="Keywords" content="chihuauhapsy,rasowe,rasy,ps\xc3\xb3w,zdj\xc4\x99cia,psy,gryfoniki,szczeni\xc4\x99ta,sprzeda\xc5\xbc,hodowla,szczeni\xc4\x99ta chihuahua na srzeda\xc5\xbc, sprzeda\xc5\xbc szczeni\xc4\x85t,chihuahuadogs,genuine,pictures,puppies,sell,CHIHUAUHA PUPPIES FOR SALE,for sale,wanted,animal husbandry">\n    <meta name="Description" content="Hodowla chihuahua i gryfonik\xc3\xb3w. Chihuahua and griffons breeding.">\n\t<meta http-equiv="cache-control" content="max-age=0" />\n\t<meta http-equiv="cache-control" content="no-cache" />\n\t<meta http-equiv="expires" content="0" />\n\t<meta http-equiv="expires" content="Tue, 01 Jan 1980 1:00:00 GMT" />\n\t<meta http-equiv="pragma" content="no-cache" />\n\n    <title>Hodowla chihuahua i gryfonik\xc3\xb3w. Chihuahua and griffons breeding.</title>\n    <link rel="stylesheet" type="text/css" href="css/styles.css">\n    <sc